In [8]:
%pip install langchain
%pip install python-dotenv
%pip install langchain-openai
%pip install beautifulsoup4
%pip install faiss-cpu

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [10]:
import os 
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_community.document_loaders import WebBaseLoader ## for web scraping
from langchain_text_splitters import RecursiveCharacterTextSplitter ## for text splitting
from langchain_openai import OpenAIEmbeddings ## for embeddings
from langchain_community.vectorstores import FAISS ## to store embeddings
load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGSMITH_API_KEY')
os.environ['LANGCHAIN_PROJECT'] = os.getenv('LANGCHAIN_PROJECT')
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
modelName = 'gpt-3.5-turbo'

In [ ]:
##Data ingestion
link = 'https://docs.smith.langchain.com/administration/tutorials/manage_spend'
loader = WebBaseLoader(link)
docs = loader.load()
docs

## divide the document
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=100)
splitted_docs = text_splitter.split_documents(docs)
splitted_docs



[Document(metadata={'source': 'https://docs.smith.langchain.com/administration/tutorials/manage_spend', 'title': 'Optimize tracing spend on LangSmith | 🦜️🛠️ LangSmith', 'description': 'Before diving into this content, it might be helpful to read the following:', 'language': 'en'}, page_content='Optimize tracing spend on LangSmith | 🦜️🛠️ LangSmith\n\n\n\n\n\n\nSkip to main contentJoin us at  Interrupt: The Agent AI Conference by LangChain on May 13 & 14 in San Francisco!API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppGet StartedObservabilityEvaluationPrompt EngineeringDeployment (LangGraph Platform)AdministrationTutorialsOptimize tracing spend on LangSmithHow-to GuidesSetupConceptual GuideSelf-hostingPricingReferenceCloud architecture and scalabilityAuthz and AuthnAuthentication methodsdata_formatsEvaluationDataset transformationsRegions FAQsdk_referenceAdministrationTutorialsOptimize tracing spend on LangSmithOn this pageOptimize tracing spend on LangSmith\nRecommended ReadingBefo

In [ ]:
embeddings = OpenAIEmbeddings()

vectorstoredb = FAISS.from_documents(splitted_docs, embeddings)
vectorstoredb


In [ ]:
## Querying the vector store
query="where short-lived traces are an order of magnitude less"
result = vectorstoredb.similarity_search(query, k=2)
result

[Document(id='77b6e695-491c-4dee-a11b-dcb0ee8048bd', metadata={'source': 'https://docs.smith.langchain.com/administration/tutorials/manage_spend', 'title': 'Optimize tracing spend on LangSmith | 🦜️🛠️ LangSmith', 'description': 'Before diving into this content, it might be helpful to read the following:', 'language': 'en'}, page_content='extension. For the sake of this tutorial, we will stick with the simplest condition, and leave more advanced filtering as an\nexercise to the user.\nnoteIf you want to keep a subset of traces for longer than 400 days for data collection purposes, you can create another run\nrule that sends some runs to a dataset of your choosing. A dataset allows you to store the trace inputs and outputs (e.g., as a key-value dataset),\nand will persist indefinitely, even after the trace gets deleted.\nSee results after 7 days\u200b\nWhile the total amount of traces per day stayed the same, the extended data retention traces was cut heavily.'),
 Document(id='c5429df6-c1

In [14]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model=modelName)
llm

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x0000025077638040>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x00000250776390C0>, root_client=<openai.OpenAI object at 0x00000250040BA260>, root_async_client=<openai.AsyncOpenAI object at 0x000002507763A410>, model_kwargs={}, openai_api_key=SecretStr('**********'))

In [ ]:
## chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """
Answer the following question based only on the provided context.
<context>
{context}
</context>
Question: {input}
    """
)

documentChain = create_stuff_documents_chain(llm,prompt)
documentChain


RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| PromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context.\n<context>\n{context}\n</context>\nQuestion: {input}\n    ')
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x0000025077638040>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x00000250776390C0>, root_client=<openai.OpenAI object at 0x00000250040BA260>, root_async_client=<openai.AsyncOpenAI object at 0x000002507763A410>, model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrOutputParser(), kwargs={}, config={'run_name': 'stuff_documents_chain'}, config_factories=[])

In [ ]:
from langchain_core.documents import Document
## i used small contect to test the model this is a cheaper way
manualDocument = Document(page_content="LangSmith Traces usage is measured per workspace, because workspaces often represent development environments (as in our example), or teams within an organization. As a LangSmith administrator, we want to understand spend granularly per each of these units. In this case where we just want to cut spend, we can focus on the environment responsible for the majority of costs first for the greatest savings.")
documentChain.invoke({
    "input": "Why is LangSmith Traces usage measured per workspace?",
    "context": [manualDocument]
})

'LangSmith Traces usage is measured per workspace because workspaces often represent development environments or teams within an organization, allowing administrators to understand spend granularly per each unit. This approach helps in identifying the environment responsible for the majority of costs, enabling greater savings when cutting spend.'

In [30]:
## retriever
from langchain.chains import create_retrieval_chain
retriever= vectorstoredb.as_retriever()
retrieverChain = create_retrieval_chain(retriever,documentChain)
retrieverChain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000025075403520>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | PromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context.\n<context>\n{context}\n</context>\nQuestion: {input}\n    ')
            | ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x0000025077638040>, async_client=<opena

In [32]:
## get the response from the llm
response = retrieverChain.invoke({
    "input": "Why is LangSmith Traces usage measured per workspace?",
    "context": [manualDocument]
})
print(response)
answer = response['answer']
print(answer)

{'input': 'Why is LangSmith Traces usage measured per workspace?', 'context': [Document(id='dade513e-002b-4030-82cf-2dde2ee07a26', metadata={'source': 'https://docs.smith.langchain.com/administration/tutorials/manage_spend', 'title': 'Optimize tracing spend on LangSmith | 🦜️🛠️ LangSmith', 'description': 'Before diving into this content, it might be helpful to read the following:', 'language': 'en'}, page_content="The first metric tracks all traces that you send to LangSmith. The second tracks all traces that also have our Extended 400 Day Data Retention.\nFor more details, see our data retention conceptual docs. Notice that these graphs look\nidentical, which will come into play later in the tutorial.\nLangSmith Traces usage is measured per workspace, because workspaces often represent development environments (as in our example),\nor teams within an organization. As a LangSmith administrator, we want to understand spend granularly per each of these units. In\nthis case where we just w

In [33]:
response['context']

[Document(id='dade513e-002b-4030-82cf-2dde2ee07a26', metadata={'source': 'https://docs.smith.langchain.com/administration/tutorials/manage_spend', 'title': 'Optimize tracing spend on LangSmith | 🦜️🛠️ LangSmith', 'description': 'Before diving into this content, it might be helpful to read the following:', 'language': 'en'}, page_content="The first metric tracks all traces that you send to LangSmith. The second tracks all traces that also have our Extended 400 Day Data Retention.\nFor more details, see our data retention conceptual docs. Notice that these graphs look\nidentical, which will come into play later in the tutorial.\nLangSmith Traces usage is measured per workspace, because workspaces often represent development environments (as in our example),\nor teams within an organization. As a LangSmith administrator, we want to understand spend granularly per each of these units. In\nthis case where we just want to cut spend, we can focus on the environment responsible for the majority